In [2]:
import folium
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely import wkt

In [6]:
def prepare_data(pid:str):    
    #load all data
    df = pd.read_csv("../../test_full_pattern.csv")
    # pick one trip to work with
    
    trip_id = df["unique_trip_vehicle_day"][0]

    one_trip_df = df[df["unique_trip_vehicle_day"]== trip_id]

    one_trip_df['location'] = one_trip_df['location'].astype(str)
    one_trip_df['geometry'] = one_trip_df['location'].apply(wkt.loads)

    one_trip_df = gpd.GeoDataFrame(one_trip_df, geometry='geometry', crs='epsg:4326')

    bus = one_trip_df[one_trip_df['typ'] == 'B']
    bus = gpd.GeoSeries(bus["geometry"])
    bus = bus.to_json()

    stop = one_trip_df[one_trip_df['typ'] == 'S']
    stop = gpd.GeoSeries(stop["geometry"])
    stop = stop.to_json()

    other = one_trip_df[one_trip_df['typ'] == 'W']
    other = gpd.GeoSeries(other["geometry"])
    other = other.to_json()

    segments_df = pd.read_parquet(f"../cta-stop-etl/out/patterns/pid_{pid}_segment.parquet")
    segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
    segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:4326')

    pattern_segments = gpd.GeoSeries(segments_df["geometry"])
    pattern_segments = pattern_segments.to_json()

    return bus, stop, other, pattern_segments

In [4]:
# Choose a route you want to find  
rt = '29'

df = pd.read_parquet("../cta-stop-etl/out/parquets/2023-11-16.parquet")
#just chooses one of the patterns from this route
pid = int(df[df['rt']==rt]['pid'].unique()[0])

#then run to get the bus location and stop times for the pattern (just looks at 100 trips)
print(f"python -m cta-stop-watch.cta-stop-etl.calculate_stop_time {pid} 100")


python -m cta-stop-watch.cta-stop-etl.calculate_stop_time 1039 100


In [ ]:
#then run and the map below
bus, stop, other, pattern_segments = prepare_data(pid)

In [8]:
m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)


# add one trip bus location
folium.GeoJson(data=bus, marker=folium.Circle(
                   radius=50,
                   fill_color="red",
                   fill_opacity=0.4,
                   color="black",
                   weight=1), 
                   name = 'One Bus Trip').add_to(m)

# add bus stops
folium.GeoJson(data=stop, marker=folium.Circle(
                   radius=30,
                   fill_color="green",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Bus Stops').add_to(m)

folium.GeoJson(data=other, marker=folium.Circle(
                   radius=10,
                   fill_color="blue",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Other Bus Routes Waypoints').add_to(m)



# add segments
folium.GeoJson(data=pattern_segments, style_function=lambda x: {"fillColor": "grey", "weight": '1'}, name = 'Route with Buffer').add_to(m)

folium.LayerControl().add_to(m)

m

In [11]:
# weird example of all the patterns for a segments can just be parts
segments = [1800., 1798., 1802., 5736., 1799., 8140., 5376.]


patterns_json = []
for seg in segments:
    seg = int(seg)
    segments_df = pd.read_parquet(f"../cta-stop-etl/out/patterns/pid_{seg}_segment.parquet")
    segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
    segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:4326')

    pattern_segments = gpd.GeoSeries(segments_df["geometry"])
    pattern_segments = pattern_segments.to_json()
    patterns_json.append(pattern_segments)

segments = [1800., 1798., 1802., 5736., 1799., 8140., 5376.]

m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)

for i, pattern in enumerate(patterns_json):
    folium.GeoJson(data=pattern, style_function=lambda x: {"weight": '1'}, name = str(segments[i])).add_to(m)

folium.LayerControl().add_to(m)

m

In [19]:
df = pd.read_csv("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/test_full_pattern.csv")

In [39]:
stops_df = gpd.read_parquet(
        "/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/patterns/pid_23279_stop.parquet"
    )

ValueError: No geometry columns are included in the columns read from
            the Parquet/Feather file.  To read this file without geometry columns,
            use pandas.read_parquet/read_feather() instead.

In [40]:
df = pd.read_parquet("/Users/austinsteinhart/Desktop/CAPP/cta-stop-watch/cta-stop-watch/cta-stop-etl/out/patterns/pid_23279_stop.parquet")
df

,seq,lat,lon,typ,stpid,stpnm,pdist,geometry,segment
0,1,41.780539,-87.586679,S,3453,63rd Street & Stony Island,0.0,POINT (-87.586679 41.780539),0
1,2,41.780600,-87.586680,W,None,None,0.0,POINT (-87.58668 41.7806),1
2,3,41.780610,-87.586450,W,None,None,0.0,POINT (-87.58645 41.78061),2
3,4,41.780190,-87.586450,W,None,None,0.0,POINT (-87.58645 41.78019),3
4,5,41.780167,-87.586555,S,5201,Stony Island & 63rd Street,239.0,POINT (-87.586555 41.780167),4
...,...,...,...,...,...,...,...,...,...
186,187,41.708689,-87.582293,W,None,None,0.0,POINT (-87.582293 41.708689),186
187,188,41.708559,-87.582295,W,None,None,0.0,POINT (-87.582295 41.708559),187
188,189,41.708497,-87.582115,W,None,None,0.0,POINT (-87.582115 41.708497),188
189,190,41.708527,-87.581436,W,None,None,0.0,POINT (-87.581436 41.708527),189


In [24]:
df['geometry'] = df['location'].astype(str).apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:26971')




In [30]:
test = gpd.GeoSeries(gdf["geometry"])
test = gdf.to_json()

m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)

tooltip = folium.GeoJsonTooltip(
    fields=["bus_stop_time"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)


folium.GeoJson(data=test, tooltip=tooltip, style_function=lambda x: {"weight": '1'}, name = str(segments[i])).add_to(m)



folium.LayerControl().add_to(m)

m